In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from imutils import face_utils
import imutils
from threading import Thread
import dlib
import cv2
import pandas as pd

In [2]:

def e_dist(pA, pB):
	return np.linalg.norm(pA - pB)

In [3]:
def eye_ratio(eye):
	
	d_V1 = e_dist(eye[1], eye[5])
	d_V2 = e_dist(eye[2], eye[4])

	
	d_H = e_dist(eye[0], eye[3])

	
	eye_ratio_val = (d_V1 + d_V2) / (2.0 * d_H)

	return eye_ratio_val

In [4]:
def max_face(face):
    w_max = 0
    index = 0
    for i, (x, y, w, h) in enumerate(face):
        if w >= w_max:
            w_max = w
            index = i
    return index
    

In [5]:
face_detect = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
landmark_detect = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [6]:
(left_eye_start, left_eye_end) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(right_eye_start, right_eye_end) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [7]:
eye_ratio_threshold = 0.22
head_ratio_threshold = 0.25
# bw_threshold = 63

In [8]:

max_sleep_frames = 2


sleep_frames = 0

In [9]:
url = 0
camera = cv2.VideoCapture(url)

y_list = []
# print('ok')
while True:
    ret, frame = camera.read()
    frame = cv2.flip(frame,1)

    if not ret:
        cv2.waitKey(10)
        
        camera.release()
        cv2.destroyAllWindows()
        break

    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    
    frame = imutils.resize(frame, width=800)

    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # (thresh, black_and_white) = cv2.threshold(gray, bw_threshold, 255, cv2.THRESH_BINARY)
    # cv2.imshow('black_and_white', black_and_white)

    
    faces = face_detect.detectMultiScale(gray, scaleFactor=1.1,	minNeighbors=5, minSize=(200, 200),	flags=cv2.CASCADE_SCALE_IMAGE)
    # faces_bw = face_detect.detectMultiScale(black_and_white, scaleFactor=1.1,	minNeighbors=5, minSize=(100, 100),	flags=cv2.CASCADE_SCALE_IMAGE)
    # print(faces)

    # if len(faces_bw) > 0:
    #     index_f = max_face(faces_bw)
    #     (x,y,w,h) = faces_bw[index_f]

    #     cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 3)

    if len(faces) > 0: 

        index_f = max_face(faces)

        (x,y,w,h) = faces[index_f]
        # print(faces[index_f])
        # for (x, y, w, h) in faces:

        
        rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 3)

        
        landmark = landmark_detect(gray, rect)
        landmark = face_utils.shape_to_np(landmark)

        
        leftEye = landmark[left_eye_start:left_eye_end]
        rightEye = landmark[right_eye_start:right_eye_end]

        left_eye_ratio = eye_ratio(leftEye)
        right_eye_ratio = eye_ratio(rightEye)

        eye_avg_ratio = (left_eye_ratio + right_eye_ratio) / 2.0

        
        left_eye_bound = cv2.convexHull(leftEye)
        right_eye_bound = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [left_eye_bound], -1, (0, 255, 0), 2)
        cv2.drawContours(frame, [right_eye_bound], -1, (0, 255, 0), 2)

        
        nose = landmark[29]
        left_head = landmark[1]
        right_head = landmark[15]

        
        cv2.line(frame, left_head, nose, (0,255,255), 2)
        cv2.line(frame, right_head, nose, (255,255,0), 2)

        
        left_head_ratio = e_dist(nose, left_head) / w
        right_head_ratio = e_dist(nose, right_head) / w

        head_avg_ratio = (left_head_ratio + right_head_ratio)/2.0

        # if left_jaw_ratio < jaw_ratio_threshold or right_jaw_ratio < jaw_ratio_threshold:
        #     print('left:',left_jaw_ratio)
        #     print('right:',right_jaw_ratio)
        #     print('-------------')

        
        if eye_avg_ratio < eye_ratio_threshold:
            sleep_frames += 1

            if sleep_frames >= max_sleep_frames:

                
                cv2.putText(frame, "LOW  -  Eye", (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                y_list.append(0)
                # st.header('LOW')

        
        else:

            if left_head_ratio < head_ratio_threshold or right_head_ratio < head_ratio_threshold:
                cv2.putText(frame, "LOW  -  Head", (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
                y_list.append(0)

            else: 

                
                sleep_frames = 0
                alarmed = False

                
                cv2.putText(frame, "EYE AVG RATIO: {:.3f}".format(eye_avg_ratio), ((x+120,y-50)),	cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                cv2.putText(frame, "HEAD AVG RATIO: {:.3f}".format(head_avg_ratio), ((x+120,y-20)),	cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                cv2.putText(frame, "HIGH!!!", (x,y-50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
                y_list.append(1)

    else: 
        cv2.putText(frame, "LOW  -  Face", (300,300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        y_list.append(0)


    cv2.imshow('Mood Detection',frame)

    k = cv2.waitKey(10) & 0xFF
    if k == 27: break

camera.release()
cv2.destroyAllWindows()

plt.plot(range(len(y_list)),y_list)

print('Tổng số phút:', round(len(y_list)/6000, 2))
print('Tổng số phút tập trung:', round(np.count_nonzero(y_list)/6000, 2))

KeyboardInterrupt: 

In [ ]:
camera.release()